In [1]:
import pandas as pd, numpy as np, pickle
mira_grn = pd.read_csv('/ocean/projects/cis240075p/skeshari/igvf/bcell2/primaryBCell/out_data/out_other_methods/ISD/ISD_matrix.chip.tsv', sep='\t',header = None, low_memory=False, index_col=0)
TFdatabase = pd.read_csv('/ix/djishnu/Swapnil/CellOracle/BCell/primaryBCell/notebooks/human TF database.csv')
TF_mira = set(mira_grn.iloc[0,0:])
TFs_in_mira = set(TFdatabase['HGNC symbol']).intersection(TF_mira)
TFs_in_mira.add('CD74')
mira_grn = mira_grn.iloc[:,list(mira_grn.iloc[0,0:].isin(TFs_in_mira))]
assert len(TFs_in_mira) == len(set(mira_grn.iloc[0,0:]))

In [4]:
import scanpy as sc
adata = sc.read_h5ad('/ocean/projects/cis240075p/skeshari/igvf/bcell2/primaryBCell/out_data/out_other_methods/ISD/adata_aggregated_ATAC.ISD.h5ad')

In [11]:
adata.obs_names

Index(['AAACAGCCAAAGCTAA-1', 'AAACAGCCAAGCCACT-3', 'AAACAGCCAAGGTGCA-1',
       'AAACAGCCAAGTTATC-1', 'AAACAGCCAATAGCCC-1', 'AAACAGCCAATTATGC-2',
       'AAACAGCCAGTTAGCC-1', 'AAACAGCCATTCAGCA-3', 'AAACATGCAAAGCGCA-1',
       'AAACATGCAAAGCTCC-1',
       ...
       'TTTGTTGGTGCCTCAC-2', 'TTTGTTGGTGTTGCAA-1', 'TTTGTTGGTTAGCGTA-1',
       'TTTGTTGGTTATCCGT-3', 'TTTGTTGGTTGACTTC-1', 'TTTGTTGGTTTACGTC-1',
       'TTTGTTGGTTTAGTCC-1', 'TTTGTTGGTTTATGGG-2', 'TTTGTTGGTTTCCTCC-3',
       'TTTGTTGGTTTGAGGC-2'],
      dtype='object', length=32418)

In [6]:
ISD_values = mira_grn.iloc[1:,:].astype(float)
# Find threshold for the ISD values
ISD_values_array = ISD_values.values.reshape(-1)
threshold = np.percentile(ISD_values_array, 50)
ISD_values = (ISD_values > threshold).astype(int).T
print(ISD_values.sum(axis=1).mean())
ISD_values['TFs'] = mira_grn.iloc[0,0:]
ISD_values = ISD_values.groupby('TFs').sum().T
TF_dictionary = {col: ISD_values.index[ISD_values[col] != 0].tolist() for col in ISD_values.columns}
TF_perturb_list = set(["BATF","BATF3","BCL6","EGR1","FOS","IKZF1","IRF1","IRF4","IRF8","JUNB","JUND","MEF2A","MEF2C","MYB","NFATC1","NTC1","NFATC2","NFIL3","NFKB1","NFKB2","PAX5","PRDM1","RUNX1","SP3","SPI1","STAT1","STAT5A","TCF12","VDR","XBP1","ZBTB7A","SPIB"])
TF_perturb_list.issubset(set(TF_dictionary.keys()))


1106.5751687798452


False

In [6]:
with open('../out_data/TF_dict_MIRA_85pct.pkl', 'wb') as f:
    pickle.dump(TF_dictionary, f)

1214.599860659545